In [94]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geocoder
import geocoder as geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 102kB 19.2MB/s ta 0:00:01
Libraries imported.


In [95]:
!wget -q -O 'london_data.json' https://en.wikipedia.org/wiki/SW_postcode_area
print('Data downloaded!')

Data downloaded!


In [96]:
london_df = pd.read_html("https://en.wikipedia.org/wiki/List_of_areas_of_London", header=0, attrs={"class":"wikitable sortable"})[0]
london_df.head()

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [2]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[3]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[3],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[3],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [97]:
london_df.shape

(533, 6)

In [98]:
london_df.columns

Index(['Location', 'London borough', 'Post town', 'Postcode district',
       'Dial code', 'OS grid ref'],
      dtype='object')

In [99]:
london_df.columns = ['Location','London_borough','Post town','Postcode district','Dial Code','OS grid ref']

In [100]:
london_df['London_borough'] =  london_df['London_borough'].apply(lambda x: x.replace('[','').replace(']','')) 
london_df['London_borough'] =  london_df['London_borough'].str.replace('\d+', '')
london_df['London_borough'] =  london_df['London_borough'].str.split(',').str[0]
london_df['Postcode district'] =  london_df['Postcode district'].str.split(',').str[0]
london_df['Postcode district'] =  london_df['Postcode district'].str.split('(').str[0]
london_df['Post town'] =  london_df['Post town'].str.split(',').str[0]

In [101]:
london_df.head()

,Location,London_borough,Post town,Postcode district,Dial Code,OS grid ref
0,Abbey Wood,Bexley,LONDON,SE2,020,TQ465785
1,Acton,Ealing,LONDON,W3,020,TQ205805
2,Addington,Croydon,CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon,CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,BEXLEY,DA5,020,TQ478728


In [102]:
london_df.to_csv('london.csv',index=False)

In [103]:
london_df.shape

(533, 6)

## Getting property data of London

In [104]:
tables = pd.read_html("https://propertydata.co.uk/cities/london",header=0)
property_df=pd.DataFrame(data=tables[0])
property_df

,Area,Avg yield,Avg price,£/sqft,5yr +/-,Explore data
0,BR1,3.3%,"£439,284",£462,+21%,Explore data
1,BR2,3.4%,"£457,502",£459,+19%,Explore data
2,BR3,3.6%,"£440,550",£489,+19%,Explore data
3,BR4,-,"£569,033",£452,+17%,Explore data
4,BR5,2.9%,"£439,343",£422,+21%,Explore data
5,BR6,3.2%,"£516,698",£463,+21%,Explore data
6,BR7,-,"£572,208",£491,+18%,Explore data
7,BR8,-,"£349,151",£354,+22%,Explore data
8,CR0,4.1%,"£347,556",£433,+18%,Explore data
9,CR2,3.7%,"£377,609",£434,+16%,Explore data


In [105]:
property_df.columns

Index(['Area', 'Avg yield', 'Avg price', '£/sqft', '5yr +/-', 'Explore data'], dtype='object')

In [106]:
property_df.drop(columns=['Avg yield','£/sqft','5yr +/-','Explore data'],inplace=True)


In [107]:
property_df.head()

,Area,Avg price
0,BR1,"£439,284"
1,BR2,"£457,502"
2,BR3,"£440,550"
3,BR4,"£569,033"
4,BR5,"£439,343"


In [108]:
property_df['Avg price'] = property_df['Avg price'].str.replace("£","")
property_df['Avg price'] = property_df['Avg price'].str.replace(',', '')
property_df['Avg price'] = pd.to_numeric(property_df['Avg price'])

In [109]:
property_df.head()

,Area,Avg price
0,BR1,439284
1,BR2,457502
2,BR3,440550
3,BR4,569033
4,BR5,439343


In [110]:
property_df.columns=['Postcode district','Avg price']

In [111]:
property_df.columns

Index(['Postcode district', 'Avg price'], dtype='object')

In [112]:
property_df.to_csv('london_postcode.csv',index=False)

# Merge the table of London Boroughs and London House Prices

In [113]:
london_data=pd.merge(london_df, property_df, how='inner', left_on='Postcode district', right_on='Postcode district')

In [114]:
london_data.columns

Index(['Location', 'London_borough', 'Post town', 'Postcode district',
       'Dial Code', 'OS grid ref', 'Avg price'],
      dtype='object')

In [115]:
london_data.head()

,Location,London_borough,Post town,Postcode district,Dial Code,OS grid ref,Avg price
0,Abbey Wood,Bexley,LONDON,SE2,020,TQ465785,340136
1,Crossness,Bexley,LONDON,SE2,020,TQ480800,340136
2,West Heath,Bexley,LONDON,SE2,020,TQ475775,340136
3,Acton,Ealing,LONDON,W3,020,TQ205805,530837
4,Addington,Croydon,CROYDON,CR0,020,TQ375645,347556


In [116]:
def getLatLong(row):
    #print('post :{}'.format(row[:]))
    #print('neigh :{}'.format(row[1]))
    # initialize your variable to None
    lat_lng_coords = None
    search_query = '{}, London,UK'.format(row)
    # loop until you get the coordinates
    try:
        while(lat_lng_coords is None):
            #g = geocoder.here(search_query,app_id=app_id,app_code=app_code)
            g = geocoder.arcgis(search_query)
            lat_lng_coords = g.latlng
            #print('FIRST')
    except IndexError:
        latitude = 0.0
        longitude = 0.0
        print('BACKUP')
        return [latitude,longitude]

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    print(latitude, longitude)
    return [latitude, longitude]

In [ ]:
coords_list = london_data['Postcode district'].apply(getLatLong).tolist()

51.492450000000076 0.12127000000003818
51.492450000000076 0.12127000000003818
51.492450000000076 0.12127000000003818
51.51324000000005 -0.2674599999999714
51.38475500000004 -0.051498623999975734
51.38475500000004 -0.051498623999975734
51.38475500000004 -0.051498623999975734
51.38475500000004 -0.051498623999975734
51.38475500000004 -0.051498623999975734
51.38475500000004 -0.051498623999975734
51.38475500000004 -0.051498623999975734
51.38475500000004 -0.051498623999975734
51.38475500000004 -0.051498623999975734
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.41009000000008 -0.05682999999993399
51.41009000000008 -0.05682999999993399
51.271725510118685 0.07470987752966636
51.271725510118685 0.07470

51.617800000000045 -0.009339999999951942
51.413275000000056 0.0874730970000428
51.413275000000056 0.0874730970000428
51.60104000000007 -0.19400999999993473
51.46095000000008 -0.13921999999996615
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.58486000000005 -0.24880999999993492
51.58486000000005 -0.24880999999993492
51.58486000000005 -0.24880999999993492
51.58486000000005 -0.24880999999993492
51.58486000000005 -0.24880999999993492
51.421700000000044 -0.2079599999999573
51.421700000000044 -0.2079599999999573
51.421700000000044 -0.2079599999999573
51.421700000000044 -0.2079599999999573
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.41087440213694 -0.291946320786896

In [39]:
london_neighborhoods = london_data.join(geo_data)
london_neighborhoods.head(10)

,Location,London_borough,Post town,Dial Code,OS grid ref,Avg price,Latitude,Longitude
BR1,Bromley,Bromley,BROMLEY,020,TQ405695,439284,NaN,NaN
BR1,Downham,Lewisham,BROMLEY,020,TQ395715,439284,NaN,NaN
BR1,Plaistow,Bromley,BROMLEY,020,TQ405705,439284,NaN,NaN
BR1,Sundridge,Bromley,BROMLEY,020,TQ406702,439284,NaN,NaN
BR1,Widmore (also Widmore Green),Bromley,BROMLEY,020,TQ411691,439284,NaN,NaN
BR2,Hayes,Bromley,BROMLEY,020,TQ405665,457502,NaN,NaN
BR2,Keston,Bromley,KESTON,01698,TQ415645,457502,NaN,NaN
BR2,Leaves Green,Bromley,KESTON,01689,TQ414616,457502,NaN,NaN
BR2,Southborough,Bromley,BROMLEY,020,TQ423676,457502,NaN,NaN
BR3,Beckenham,Bromley,BECKENHAM,020,TQ375695,440550,NaN,NaN


In [40]:
london_neighborhoods.drop(columns=['Post town','Dial Code','OS grid ref'],inplace=True)

In [41]:
london_neighborhoods.head()

,Location,London_borough,Avg price,Latitude,Longitude
BR1,Bromley,Bromley,439284,NaN,NaN
BR1,Downham,Lewisham,439284,NaN,NaN
BR1,Plaistow,Bromley,439284,NaN,NaN
BR1,Sundridge,Bromley,439284,NaN,NaN
BR1,Widmore (also Widmore Green),Bromley,439284,NaN,NaN


In [47]:
london_neighborhoods = london_neighborhoods.rename(columns={'Location': 'Neighborhood', 'London_borough': 'Borough'})

In [48]:
london_neighborhoods.head()

,Neighborhood,Borough,Avg price,Latitude,Longitude
BR1,Bromley,Bromley,439284,NaN,NaN
BR1,Downham,Lewisham,439284,NaN,NaN
BR1,Plaistow,Bromley,439284,NaN,NaN
BR1,Sundridge,Bromley,439284,NaN,NaN
BR1,Widmore (also Widmore Green),Bromley,439284,NaN,NaN


In [49]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(london_neighborhoods['Borough'].unique()),
        london_neighborhoods.shape[0]
    )
)

The dataframe has 35 boroughs and 507 neighborhoods.


In [50]:
address = 'London, LN'

geolocator = Nominatim(user_agent="ln_explorer")
location = geolocator.geocode(address)
london_latitude = location.latitude
london_longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(london_latitude, london_longitude))

The geograpical coordinate of London are 43.6342758, -123.0909754.


# Create a map of London with neighborhoods